# Day 8 - Lab 1: Vision-Enabled UI/UX Agents

**Objective:** Use multi-modal vision models to generate a frontend UI from a design image, and then use a second agent to perform an automated design review.

**Estimated Time:** 90 minutes

**Introduction:**
Welcome to Day 8! Today, we'll explore one of the most exciting advancements in AI: vision. We will use a vision-capable model to act as a frontend developer, translating a design image directly into code. Then, we will create a second "UI/UX Critic" agent to automate the design review process, demonstrating a complete, AI-assisted frontend workflow.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

For this lab, we need to ensure we are using a vision-capable model. We will configure our `utils.py` helper to use a model like OpenAI's `gpt-4o` or Google's `gemini-2.5-pro`.

**Model Selection:**
This lab requires a vision-capable model. Excellent choices include `gpt-4o`, `gemini-2.5-pro`, or `deepseek-ai/DeepSeek-VL2`.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_vision_completion()`: A specialized function to send an image and a text prompt to a vision model.
- `get_completion()`: To send text-only prompts for the refactoring step.
- `save_artifact()`: To save the generated code and the design review.

In [1]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_vision_completion, get_completion, save_artifact, clean_llm_output
from IPython.display import Image, display, Code, Markdown

# Ensure you select an image generation-capable model
image_client, image_model_name, image_api_provider = setup_llm_client(model_name="dall-e-3")

# Ensure you select a vision-capable model
vision_client, vision_model_name, vision_api_provider = setup_llm_client(model_name="gpt-4o")

# Use the vision-capable client as the default client used later in the notebook
client, model_name, api_provider = vision_client, vision_model_name, vision_api_provider

if not model_name:
    print("Could not set up a valid LLM client. Please check your .env file and utils.py configuration.")

✅ LLM Client configured: Using 'openai' with model 'dall-e-3'
✅ LLM Client configured: Using 'openai' with model 'gpt-4o'
✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Design Screenshot

This is the design we want our AI agent to build. It's a simple login form component. We will provide the URL to this image directly to the model.

In [2]:
login_form_url = "https://i.imgur.com/XcJ6H9M.png"
display(Image(url=login_form_url))

## Step 3: The Challenges

### Challenge 1 (Foundational): Generating a Monolithic UI Component

**Task:** Use the vision model to generate a single, self-contained React component that replicates the design from the image.

> **Tip for Vision Prompts:** Be specific about the output format. Telling the model you want 'React' and 'Tailwind CSS' is crucial. The more specific your technical constraints, the better the generated code will be.

**Instructions:**
1.  Create a prompt that asks the vision model to act as an expert frontend developer.
2.  The prompt should instruct the model to analyze the image at the provided URL.
3.  Ask it to generate a single React component using Tailwind CSS for styling.
4.  The output should be a single block of JSX code.

**Expected Quality:** A single file's worth of React code that, when rendered, visually approximates the login form in the screenshot.

In [3]:

# TODO: Write a prompt to generate a single React component from the image.
generate_ui_prompt = f"""
You are an expert frontend developer specializing in React and Tailwind CSS.

Your task is to analyze the provided image of a login form and write the code for a single, self-contained React component that accurately replicates its design and layout.

**Requirements:**
- Use functional components.
- Use Tailwind CSS for all styling. Do not use custom CSS or style tags.
- Make sure the component is accessible, using appropriate HTML tags and attributes.
- The output should be only the raw JSX code for the component.
"""

print("--- Generating Monolithic UI Component ---")
if model_name:
    generated_monolithic_code = get_vision_completion(generate_ui_prompt, login_form_url, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_monolithic_code, language='jsx')
    display(Code(cleaned_code, language='javascript'))
    save_artifact(cleaned_code, "app/day8_login_monolithic.jsx")
else:
    print("Skipping UI generation because no valid model is configured.")
    cleaned_code = ""

--- Generating Monolithic UI Component ---


import React from 'react';

function LoginForm() {
  return (
    <div className="min-h-screen flex items-center justify-center bg-gray-100">
      <div className="bg-white p-8 rounded-lg shadow-lg max-w-md w-full">
        <h2 className="text-2xl font-bold mb-6 text-center">Login</h2>
        <form>
          <div className="mb-4">
            <label
              htmlFor="email"
              className="block text-gray-700 text-sm font-bold mb-2"
            >
              Email
            </label>
            <input
              type="email"
              id="email"
              name="email"
              className="shadow appearance-none border rounded w-full py-2 px-3 text-gray-700 leading-tight focus:outline-none focus:shadow-outline"
              placeholder="Enter your email"
            />
          </div>
          <div className="mb-4">
            <label
              htmlFor="password"
              className="block text-gray-700 text-sm font-bold mb-2"
            >
              Password
            </label>
            <input
              type="password"
              id="password"
              name="password"
              className="shadow appearance-none border rounded w-full py-2 px-3 text-gray-700 mb-3 leading-tight focus:outline-none focus:shadow-outline"
              placeholder="Enter your password"
            />
          </div>
          <div className="flex items-center justify-between">
            <button
              type="submit"
              className="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded focus:outline-none focus:shadow-outline"
            >
              Sign In
            </button>
            <a
              className="inline-block align-baseline font-bold text-sm text-blue-500 hover:text-blue-800"
              href="#"
            >
              Forgot Password?
            </a>
          </div>
        </form>
      </div>
    </div>
  );
}

export default LoginForm;

✅ Successfully saved artifact to: app/day8_login_monolithic.jsx


### Challenge 2 (Intermediate): Refactoring into Reusable Components

**Task:** A single, large component is not good practice. Now, prompt the LLM to refactor the monolithic code it just generated into smaller, reusable sub-components.

**Instructions:**
1.  Create a new prompt.
2.  Provide the monolithic JSX code from the previous step as context.
3.  Instruct the LLM to act as a senior frontend developer who champions clean code.
4.  Ask it to refactor the code by creating smaller, reusable components (e.g., `<StyledButton>`, `<InputWithIcon>`).
5.  The final output should be the complete code with the new, smaller components defined and used within the main `Login` component.

**Expected Quality:** A well-structured React file that demonstrates the component-based architecture, which is a fundamental best practice in modern frontend development.

In [4]:
refactor_ui_prompt = f"""
You are a senior frontend developer who champions clean, component-based architecture.

The following React code is a single, monolithic component. Your task is to refactor it.

**Refactoring Requirements:**
1.  Create smaller, reusable sub-components where appropriate. For example, the button and the input fields are good candidates for their own components (e.g., `InputWithIcon`, `LoginButton`).
2.  The final output should be a single file containing the definitions for the new, smaller components and the main `LoginForm` component that uses them.
3.  Ensure the final visual output remains identical to the original.

**Original Code:**
```jsx
{cleaned_code}
```

Output only the raw, complete, refactored JSX code.
"""

print("--- Refactoring UI into Components ---")
cleaned_refactored_code = ""
if 'cleaned_code' in locals() and cleaned_code:
    refactored_code = get_completion(refactor_ui_prompt, client, model_name, api_provider)
    cleaned_refactored_code = clean_llm_output(refactored_code, language='jsx')
    display(Code(cleaned_refactored_code, language='javascript'))
    save_artifact(cleaned_refactored_code, "app/day8_login_refactored.jsx")
else:
    print("Skipping refactoring because monolithic code was not generated.")

--- Refactoring UI into Components ---


import React from 'react';

// Sub-component for input fields with label
function InputWithIcon({ type, id, name, placeholder, label }) {
  return (
    <div className="mb-4">
      <label
        htmlFor={id}
        className="block text-gray-700 text-sm font-bold mb-2"
      >
        {label}
      </label>
      <input
        type={type}
        id={id}
        name={name}
        className="shadow appearance-none border rounded w-full py-2 px-3 text-gray-700 leading-tight focus:outline-none focus:shadow-outline"
        placeholder={placeholder}
      />
    </div>
  );
}

// Sub-component for the login button
function LoginButton({ children }) {
  return (
    <button
      type="submit"
      className="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded focus:outline-none focus:shadow-outline"
    >
      {children}
    </button>
  );
}

// Main LoginForm component
function LoginForm() {
  return (
    <div className="min-h-screen flex items-center justify-center bg-gray-100">
      <div className="bg-white p-8 rounded-lg shadow-lg max-w-md w-full">
        <h2 className="text-2xl font-bold mb-6 text-center">Login</h2>
        <form>
          <InputWithIcon
            type="email"
            id="email"
            name="email"
            placeholder="Enter your email"
            label="Email"
          />
          <InputWithIcon
            type="password"
            id="password"
            name="password"
            placeholder="Enter your password"
            label="Password"
          />
          <div className="flex items-center justify-between">
            <LoginButton>Sign In</LoginButton>
            <a
              className="inline-block align-baseline font-bold text-sm text-blue-500 hover:text-blue-800"
              href="#"
            >
              Forgot Password?
            </a>
          </div>
        </form>
      </div>
    </div>
  );
}

export default LoginForm;

✅ Successfully saved artifact to: app/day8_login_refactored.jsx


### Challenge 3 (Advanced): The AI UI/UX Critic Agent

**Task:** Create a new "UI/UX Critic" agent. This agent will be given both the original design image and the generated code, and its job is to perform an automated design review.

**Instructions:**
1.  Create a final, complex prompt for a new agent.
2.  The prompt should instruct the agent to act as a meticulous UI/UX designer.
3.  Provide the agent with two pieces of context: the URL of the original design image and the final, refactored React code.
4.  The agent's task is to compare the code's likely rendered output to the design image and list any visual inconsistencies in spacing, font size, color, or layout.

**Expected Quality:** A critical design review in markdown format. This demonstrates a powerful AI-on-AI workflow, where one AI generates work and another AI validates it, automating a time-consuming QA step.

In [5]:
critic_agent_prompt = f"""
You are a meticulous UI/UX designer with a sharp eye for detail. Your task is to perform a design review.

You will be given two things:
1.  An image of the original design mockup.
2.  The React code that was generated to implement that design.

Your job is to compare the likely visual output of the code to the original design image. Identify and list any visual inconsistencies, no matter how small. Pay close attention to:
- Spacing and padding
- Font sizes and weights
- Color hex codes
- Border radius and shadows
- Overall layout and alignment

**Generated Code:**
```jsx
{cleaned_refactored_code}
```

Provide your feedback as a markdown-formatted list.
"""

print("--- Invoking UI/UX Critic Agent ---")
if cleaned_refactored_code:
    design_review = get_vision_completion(critic_agent_prompt, login_form_url, client, model_name, api_provider)
    display(Markdown(design_review))
    save_artifact(design_review, "artifacts/design_review.md")
else:
    print("Skipping critic agent because refactored code is not available.")

--- Invoking UI/UX Critic Agent ---


To accurately perform a design review, it’s essential to compare the likely visual output of the provided React code against an original design mockup. Below is a detailed list of potential inconsistencies:

1. **General Layout and Alignment:**
   - The original design appears to display a dashboard with multiple panels and widgets, whereas the React implementation describes a single login form. They serve different purposes and contexts.

2. **Color Scheme:**
   - The original design uses a muted color palette with blue accents, specific whites, and subtle grays. The React code uses `bg-blue-500` and `bg-blue-700`, which may not match the exact blue shades in the mockup.
   - Background color in the React code is `bg-gray-100`, which might not match the precise lighter gray tone in the mockup.

3. **Font Sizes and Weights:**
   - The font in the design mockup appears to be larger and potentially heavier in some areas, such as headings or accent elements. 
   - The React code sets text sizes using utilities like `text-2xl` and `text-sm`, which may not reflect the sizes in the design.

4. **Buttons and Interactions:**
   - The button styles in the React code (`bg-blue-500`, `hover:bg-blue-700`, `text-white`) might not match the rounded, shadowed buttons seen in the design mockup, especially regarding the subtleties of hover effects and active states.

5. **Spacing and Padding:**
   - The mockup seems to have more generous spacing and padding around elements, while the React code uses `py-2`, `px-3`, which may be tighter.
   - Margins like `mb-4` and `mb-6` may not align with the more spacious design seen in the mockup.

6. **Borders and Shadows:**
   - The React code uses `shadow-lg` and `rounded-lg`, but the shadow effects and border radius in the mockup are subtle and may differ in size or opacity.

7. **Icons and Imagery:**
   - The React code does not include any avatar or icons, while the design mockup displays a prominent user avatar and side panel icons.

8. **Other UI Elements:**
   - No corresponding task list, pie chart, or dashboard features are included in the React code but are present in the mockup.
   - The elements like "Today's Tasks" and circular progress bars are not represented in the code.

This feedback highlights the significant visual and functional differences between the React code and the design mockup. Adjustments would need to focus on accurately matching the visual styles and functional components present in the mockup.

✅ Successfully saved artifact to: artifacts/design_review.md


## Lab Conclusion

Fantastic! You have completed a full, end-to-end frontend development workflow using multiple AI agents. You used a vision-powered agent to generate code from a design, a refactoring agent to improve the code's structure, and a critic agent to perform an automated design review. This powerful combination of skills can dramatically accelerate the process of turning visual ideas into functional user interfaces.

> **Key Takeaway:** The workflow of **Generate -> Refactor -> Critique** is a powerful AI-assisted development pattern. Using specialized agents for each step allows you to rapidly create a first draft, improve its quality, and then automatically check it for correctness, significantly speeding up the iteration cycle.